In [2]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
url = 'http://www.op.gg/ranking/ladder/'
resp = requests.get(url)
bs = BeautifulSoup(resp.text, 'html.parser')
users = bs.findAll("td", {"class": "ranking-table__cell ranking-table__cell--summoner"})
# print(bs)
# print(users)

In [4]:
# get user list
user_list = []
for line in users:
    line = str(line)
    line = line[line.find("<span>"):line.find("</span>")].replace("<span>","")
    user_list.append(line)
user_list[:5]

['deft123456', 'Tar2an', 'SKT T1 Untara', '11 14 9 7 8 20', 'Longzhu Cuzz']

In [9]:
count = 1; # start number
match_number_start = 1 # 시작할 match number , 보통 (count-1)*20+1
start_number = 51 # 시작할 위치

In [5]:
def match_data_load(user_name, match_number_start):
    file_name = "user_kda_{0}".format(user_name)
    
    driver = webdriver.Chrome('D:\chromedriver')
    driver.implicitly_wait(3)
    driver.get('http://www.op.gg/summoner/userName={0}'.format(user_name))
    
    driver.find_element_by_xpath('//a[@data-type="soloranked"]').click()
    
    game_detail_button = driver.find_elements_by_css_selector('a.Button.MatchDetail')
    for a in game_detail_button:
        a.click()
        time.sleep(1)
        
    html = driver.page_source
    PlayerData = BeautifulSoup(html, 'html.parser')
#     print(PlayerData.prettify())
    team_raw = PlayerData.find_all('tbody', {'class':'Content'})
    game_raw = PlayerData.find_all('thead', {'class':'Header'})

    win_lose = []
    champion_list = []
    name_list = []
    kill_list = []
    death_list = []
    assist_list = []

    for gline in game_raw:
        game_result_raw = gline.find_all('span', {'class':'GameResult'})

        game_result = []
        for gresult in game_result_raw:
            gresult = gresult.text.strip()
            game_result.append(gresult)
        win_lose.append(game_result)

    for line in team_raw:
        teamdata = []
        team_champion_raw = line.find_all('td', {'class':'ChampionImage Cell'})
        team_name_raw = line.find_all('td', {'class':'SummonerName Cell'})
        team_kill_raw = line.find_all('span',{'class':'Kill'})
        team_death_raw = line.find_all('span',{'class':'Death'})
        team_assist_raw = line.find_all('span',{'class':'Assist'})

        team_champion = []
        for champ in team_champion_raw:
            champ = champ.text.strip()
            champ = champ[:champ.find('\n')]
            team_champion.append(champ)
        champion_list.append(team_champion)

        team_name = []
        for name in team_name_raw:
            name = name.text.strip()
            team_name.append(name)
        name_list.append(team_name)

        team_kill = []
        team_death = []
        team_assist = []
        team_kill_participation = []

        for kills in team_kill_raw:
            kills = kills.text.strip()
            team_kill.append(kills)
        kill_list.append(team_kill)

        for deaths in team_death_raw:
            deaths = deaths.text.strip()
            team_death.append(deaths)
        death_list.append(team_death)

        for assists in team_assist_raw:
            assists = assists.text.strip()
            team_assist.append(assists)
        assist_list.append(team_assist)

    win_lose = win_lose[1:]

    KDAsorted = []
    matchorder = 1
    matchnumber = match_number_start
    for order in range(int((len(win_lose)+1)/2)):
        matchnumber = matchnumber-1
        for i in range(5):
            team_member = []
            team_member.append(matchorder)
            team_member.append(win_lose[matchnumber][0])
            team_member.append(champion_list[matchnumber][i])
            team_member.append(name_list[matchnumber][i])
            team_member.append(int(kill_list[matchnumber][i]))
            team_member.append(int(death_list[matchnumber][i]))
            team_member.append(int(assist_list[matchnumber][i]))
            KDAsorted.append(team_member)
        matchnumber += 1
        for i in range(5):
            team_member = []
            team_member.append(matchorder)
            team_member.append(win_lose[matchnumber][0])
            team_member.append(champion_list[matchnumber][i])
            team_member.append(name_list[matchnumber][i])
            team_member.append(int(kill_list[matchnumber][i]))
            team_member.append(int(death_list[matchnumber][i]))
            team_member.append(int(assist_list[matchnumber][i]))
            KDAsorted.append(team_member)
        matchnumber += 2
        matchorder += 1

    kda_data = pd.DataFrame(KDAsorted, columns=['MatchNumber','GameResult', 'Champion', 'Nickname', 'Kill', 'Death', 'Assist'])
    kda_data['KDA'] = (kda_data['Kill'] + kda_data['Assist'])/ kda_data['Death']
    kda_data.head()
    
    kda_data.to_csv(file_name, encoding="utf8")
    return matchorder

In [ ]:
for user_name in user_list:
    if count < start_number:
        count += 1
        continue
    match_number_start = match_data_load(user_name, match_number_start)
    print("complete!",count, match_number_start)
    count += 1